## Train model identify if two images are the same turtle

Try resnet based, also fine tune the last conv layer

### Import libraries

In [1]:
import os
import glob
import json
from os.path import join

import torchvision.datasets as datasets
import torchvision.transforms as transforms
import timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import random

from ShellRec.data_utils.prepare_photos import get_img_graph, split_graph
from ShellRec.model import TurtleDiffConvPool 
from ShellRec.dataset import TurtlePair 
from ShellRec.train import train

In [2]:
random.seed(42)

### Preprocessing photo files

In [3]:
all_train = get_img_graph(path = "../dataset/BoxTurtle", drop_p=[0.99,0])
split_graph(all_train, save_path = "../dataset")
_ = get_img_graph(path = "../dataset/BoxTurtle_holdout", 
                  file_to_save = "../dataset/BoxTurtle_holdout.json")

### Prepare torch dataset

In [3]:
## image transformations
transform_train = timm.data.create_transform(224, is_training = True, 
                                   auto_augment = "rand-m9-mstd0.5")


transform_test = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[.5,.5,.5], 
                             std=[0.5, 0.5, 0.5])
])

train_set = TurtlePair(data_file='../dataset/train.json',transform=transform_train)
val_set = TurtlePair(data_file='../dataset/val.json',transform=transform_test)

# Set up datasets and dataloaders
train_loader = DataLoader( TurtlePair(data_file='../dataset/train.json', 
                                             transform=transform_train), 
                                             batch_size = 32)
val_loader = DataLoader( TurtlePair(data_file='../dataset/val.json', 
                                           transform=transform_test), 
                                           batch_size = 32)

### training

Get the model set up, use a pretrained model as backbone, and add a new head to it.

In [4]:
torch.hub.set_dir('../pretrained/')

In [5]:
#device = "cpu" #
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')   

model = TurtleDiffConvPool() # use a vit backbone
#model = TurtleDiff('resnet50') # use a resnet backbone
optimizer = optim.Adam(model.parameters(), lr=5e-4)
criterion = nn.CrossEntropyLoss()
num_epochs = 10

Training loop

In [ ]:
loss_list = train(model, optimizer, criterion, train_loader, val_loader, device, num_epochs,
      save_path = "../pretrained")

Epoch: 1/10


100%|██████████| 39/39 [07:16<00:00, 11.20s/it]


Epoch: 1/10, Training Loss: 0.6483


100%|██████████| 5/5 [00:56<00:00, 11.37s/it]


Epoch: 1/10, Validation Accuracy: 44.74%
Epoch: 2/10


...

100%|██████████| 39/39 [07:26<00:00, 11.46s/it]


Epoch: 9/10, Training Loss: 1.0578


100%|██████████| 5/5 [00:56<00:00, 11.28s/it]


Epoch: 9/10, Validation Accuracy: 75.00%
Epoch: 10/10


 38%|███▊      | 15/39 [02:49<04:31, 11.33s/it]